In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy import stats
from plotly.subplots import make_subplots
import math
import importlib

In [25]:
from fipy import fidata

tickers = [
    {
        "name": "spx",
        "symbol": "^SPX",
        "start": "1920-01-01",
        "period": "1day"
    },
    {
        "name": "vbmfx",
        "symbol": "VBMFX",
        "start": "1920-01-01",
        "period": "1day"
    }
]

fidata.fetch_tickers(tickers)


/home/rennochj/workspace/ml-play/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed
/home/rennochj/workspace/ml-play/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

[*********************100%%**********************]  1 of 1 completed


In [18]:
YEARS = 30
MONTHS_PER_YEAR = 12
DAYS_PER_MONTH = 21
CURRENT_VALUE = 2_500_000
# EPSILON = 1_000_000 / CURRENT_VALUE
# EPSILON = CURRENT_VALUE / CURRENT_VALUE
EPSILON = 0 / CURRENT_VALUE

N_PERIODS = YEARS * MONTHS_PER_YEAR * DAYS_PER_MONTH

F = 1.035 ** (1.0 / MONTHS_PER_YEAR / DAYS_PER_MONTH)

# rates = data["rate"].tail(N_PERIODS).to_numpy()
rates = dist.rvs(size=N_PERIODS)

kappa = 1.0
rho = rates[0]
gamma = []

for i in range(1, N_PERIODS):
    kappa = rates[i] * kappa + math.pow(F, i)
    rho *= rates[i]
    gamma.append((rho - EPSILON) / kappa)

print(gamma[-1] * DAYS_PER_MONTH * MONTHS_PER_YEAR)

OUTFLOW = gamma[-1] * CURRENT_VALUE

adjustment = 0.0
value = CURRENT_VALUE + adjustment

outflows = []
values = []

for i in range(N_PERIODS):
    outflow = OUTFLOW * math.pow(F, i)
    outflows.append(outflow)
    value = rates[i] * value - ( outflow + adjustment )
    values.append(value)

# print(f"{adjustment * DAYS_PER_MONTH * MONTHS_PER_YEAR}: {outflow}: {value:,}: {sum(outflows):,}, {value/sum(outflows):,}")

fig = make_subplots()
fig.add_trace(
    go.Scatter(y=values)
)


0.04617261713940552


In [30]:
from fipy import forecast

importlib.reload(fidata)
importlib.reload(forecast)

YEARS = 30
MONTHS_PER_YEAR = 12
DAYS_PER_MONTH = 21
CURRENT_VALUE = 2_500_000
# EPSILON = 1_000_000 / CURRENT_VALUE
EPSILON = 0 / CURRENT_VALUE
N_PERIODS = YEARS * MONTHS_PER_YEAR * DAYS_PER_MONTH
N_ITERATIONS = 10000
F = 1.025 ** (1.0 / MONTHS_PER_YEAR / DAYS_PER_MONTH )

dist = fidata.load_model("data/vbmfx.feather")

gamma = forecast.outflow_ratio(dist, EPSILON, F, N_PERIODS, N_ITERATIONS)
gamma = [g * MONTHS_PER_YEAR * DAYS_PER_MONTH for g in gamma]

results = np.histogram(gamma, bins=50, density=False)
results_dist = stats.rv_histogram(results, density=False)

print(results_dist.ppf(0.2))

fig = make_subplots()
fig.add_trace(
    go.Scatter(x=results[1], y=results[0])
)


0.04238328040870838
